In [1]:
import pandas as pd
import torch
import torch.nn as nn
import pickle
import os
import json
import numpy as np # numpy import 추가
from tqdm import tqdm

# --- 경로 설정 ---
DATA_DIR = r"C:\Users\EL040\Desktop\MS_3rd-Project\Project\data"
MODEL_DIR = r"C:\Users\EL040\Desktop\MS_3rd-Project\Project\models"
STAGE1_MODEL_PATH = os.path.join(MODEL_DIR, "base_autoencoder.pt")
STAGE2_MODEL_PATH = os.path.join(MODEL_DIR, "stage2_classifier.pkl")
# ---

# 1. 모델 로드
# 1-1. Stage 1: Autoencoder 모델 구조 정의 (모델 훈련 총정리.txt 기반)
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(45, 256), nn.ReLU(),
            nn.Linear(256, 128), nn.ReLU(),
            nn.Linear(128, 32)
        )
        self.decoder = nn.Sequential(
            nn.Linear(32, 128), nn.ReLU(),
            nn.Linear(128, 256), nn.ReLU(),
            nn.Linear(256, 45)
        )
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Autoencoder 모델 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ae = Autoencoder()
model_ae.load_state_dict(torch.load(STAGE1_MODEL_PATH, map_location=device))
model_ae.to(device)
model_ae.eval()
print(f"1단계 Autoencoder 모델 로드 완료: {STAGE1_MODEL_PATH}")

# 1-2. Stage 2: Classifier 모델 및 임계값 로드
with open(STAGE2_MODEL_PATH, 'rb') as f:
    stage2_data = pickle.load(f)
model_clf = stage2_data['pipeline']
THRESHOLD = stage2_data['threshold']
print(f"2단계 Classifier 모델 로드 완료: {STAGE2_MODEL_PATH}")
print(f"적용될 임계값(Threshold): {THRESHOLD}")

# 2. 데이터 로드
json_files = [f for f in os.listdir(DATA_DIR) if f.endswith('.json')]
all_logs = []
for file in tqdm(json_files, desc="JSON 파일 로딩 중"):
    with open(os.path.join(DATA_DIR, file), 'r', encoding='utf-8') as f:
        for line in f:
            try:
                all_logs.append(json.loads(line))
            except json.JSONDecodeError:
                # 가끔 파일 끝에 비어있는 라인이 있을 경우 대비
                continue
df_raw = pd.DataFrame(all_logs)
print(f"총 {len(df_raw)}개의 로그를 로드했습니다.")


# 3. 전처리 함수 (!!! 사용자가 직접 채워야 하는 부분 !!!)
def preprocess_to_features(df):
    """
    원본 데이터프레임을 입력받아 1단계, 2단계 모델에 필요한 피처를 생성합니다.
    이 함수는 '02_feature_engineering.ipynb'의 로직을 기반으로 완성해야 합니다.
    
    Returns:
        - df_processed: 2단계 모델용 피처(버킷 등)가 포함된 데이터프레임
        - features_45d: 1단계 모델용 45차원 수치형 피처 (numpy array)
    """
    print("주의: 전처리 함수(preprocess_to_features)는 사용자의 실제 로직으로 채워져야 합니다.")
    
    # --- 예시 코드 (반드시 실제 코드로 교체 필요) ---
    df_processed = df.copy()
    
    # 2단계 모델용 피처 생성 (예시)
    df_processed['actor_login_bucket'] = df['actor'].apply(lambda x: x.get('login', '<EMPTY>')).astype(str).str[0]
    df_processed['repo_bucket'] = df['repo'].apply(lambda x: x.get('name', '<EMPTY>')).astype(str).str.split('/').str[0]
    df_processed['org_bucket'] = df.get('org', pd.Series(dtype=object)).apply(
        lambda x: x.get('login', '<EMPTY>') if isinstance(x, dict) else '<EMPTY>'
    ).astype(str)
    
    # 1단계 모델용 45차원 피처 생성 (예시 - 랜덤 더미 데이터)
    # 이 부분에 실제 45차원 피처 생성 로직이 들어가야 합니다.
    num_samples = len(df)
    features_45d = np.random.rand(num_samples, 45)
    # -----------------------------------------
    
    return df_processed, features_45d

df_processed, features_45d = preprocess_to_features(df_raw)


# 4. 1단계 추론: Anomaly Score 계산
print("1단계 추론을 시작합니다 (Anomaly Score 계산)...")
features_tensor = torch.FloatTensor(features_45d).to(device)

# 메모리 문제를 방지하기 위해 데이터를 작은 배치로 나누어 처리
batch_size = 8192
anomaly_scores = []
with torch.no_grad():
    for i in tqdm(range(0, len(features_tensor), batch_size), desc="Anomaly Score 계산 중"):
        batch = features_tensor[i:i+batch_size]
        reconstructions = model_ae(batch)
        mse = nn.functional.mse_loss(reconstructions, batch, reduction='none')
        batch_scores = torch.mean(mse, dim=1).cpu().numpy()
        anomaly_scores.extend(batch_scores)
        
df_processed['anomaly_score'] = anomaly_scores


# 5. 2단계 추론: 최종 이상치(0/1) 예측
print("2단계 추론을 시작합니다 (최종 예측)...")
features_for_stage2 = ['anomaly_score', 'actor_login_bucket', 'repo_bucket', 'org_bucket']
X_test = df_processed[features_for_stage2]

# 확률 예측
probabilities = model_clf.predict_proba(X_test)[:, 1]

# 임계값을 기준으로 0 또는 1로 최종 예측
df_processed['prediction'] = (probabilities >= THRESHOLD).astype(int)

# 6. 결과 저장
output_path = os.path.join(DATA_DIR, "prediction_results.csv")
# 원본 로그의 주요 정보와 예측 결과만 선택하여 저장
final_columns = ['id', 'type', 'created_at', 'actor', 'repo', 'org', 'anomaly_score', 'prediction']
# df_processed에 없는 컬럼은 제외
final_columns_exist = [col for col in final_columns if col in df_processed.columns] 

df_processed[final_columns_exist].to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"최종 결과가 {output_path} 에 저장되었습니다.")
print("각 로그의 마지막 'prediction' 컬럼에서 1이면 이상치, 0이면 정상입니다.")
print("\n--- 탐지 결과 요약 ---")
print(df_processed['prediction'].value_counts())

1단계 Autoencoder 모델 로드 완료: C:\Users\EL040\Desktop\MS_3rd-Project\Project\models\base_autoencoder.pt


c:\Users\EL040\anaconda3\envs\guardian_poc\lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\EL040\anaconda3\envs\guardian_poc\lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\EL040\anaconda3\envs\guardian_poc\lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.6.1 w

UnpicklingError: invalid load key, '\x07'.